<a href="https://colab.research.google.com/github/MarianaGCosta/Lematizacao-do-portugues/blob/main/Lematiza%C3%A7%C3%A3o_em_dados_do_portugu%C3%AAs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lematização em dados do português

Esse notebook aborda o processamento sintático de textos em português brasileiro em Python 3.10 e abrange as etapas de importação, leitura, normalização, tokenização e lematização. O código foi desenvolvido como parte de um relatório para a disciplina de **Recuperação da Informação**, do curso de Ciência da Computação da Universidade Federal do Rio de Janeiro (UFRJ), ministrado pela professora Giseli Rabello em 2024 com apoio do monitor Henrique Fernandes Rodrigues. O dataset analisado pode ser encontrado em [D&G UFF](https://deg.uff.br/corpus-dg/).

**Mariana Gonçalves da Costa** [Programa de Pós-Graduação em Informática/UFRJ]


Last updated: 12 December 2024

---

## Importação das bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import os
import nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir("/content/drive/MyDrive/Corpus DeG")

print(os.getcwd())  # Imprime o diretório de trabalho atual

Mounted at /content/drive
/content/drive/MyDrive/Corpus DeG


In [ ]:
filepath = '/content/drive/MyDrive/Corpus DeG'

## Processamento do Texto

In [ ]:
def read_files(file):
  """ Lê o conteúdo de um arquivo .txt """

  with open(file, 'r', encoding='latin-1') as file:
    file = file.read()

    return file

In [ ]:
# Lê o conteúdo em 'rio_grande.txt'
rio_grande = read_files('rio_grande.txt')

In [ ]:
stopwordsdict = {
    "artigos": ["a", "o", "as", "os", "um", "uns", "uma", "umas"],
    "pronomes": ["eu", "tu", "ele", "ela", "nós", "vós", "eles", "elas",
                "me", "te", "lhe", "o", "a", "se", "nos", "vos", "lhes", "os", "as", "se",
                "meu", "minha", "teus", "tuas", "seu", "sua", "seus", "suas", "nosso","nossa",
                "nossos", "nossas", "dela", "dele", "deles", "delas", "que", "qual", "quais", "quem",
                "cujo", "cuja", "cujos", "cujas", "este", "esta", "estes", "estas",
                "esse", "essa", "esses", "essas", "aquele", "isto", "isso", "aquilo",
                "aquela", "aqueles", "aquelas"],
    "preposicoes": ["com", "sem", "para", "de", "do", "da", "dos", "das", "em",
                    "no", "na", "nos", "nas", "num" "por", "que", "ao", "à"],
    "adverbios": ["assim", "aqui", "lá", "cá", "já", "logo", "muito", "tão",
                  "tanto", "quase", "sim", "não"],
    "conjuncoes": ["e", "mas", "entretanto", "porém", "pois", "porque", "ora", "ou", "logo",
                  "como", "seja", "todavia", "nem", "desde"],
    "verbos": ["ser", "sido", "sendo", "sou", "és", "é", "somos", "sois", "são",
              "era", "eras", "éramos", "éreis", "eram",
              "fui", "foste", "foi", "fomos", "fostes", "foram", "fora",
              "seja", "sejas", "sejamos", "sejais", "sejam", "fosse", "fôssemos", "fossem",
              "for", "formos", "forem",

              "estar", "estando", "está", "estado"
              "estou", "estás", "estamos", "estais", "estão",
              "estava", "estavas", "estávamos", "estáveis", "estavam",
              "estive", "estiveste", "esteve", "estivemos", "estivestes", "estiveram",
              "estivera", "estiveras", "estivéramos", "estivéreis", "estiveram",
              "estarei", "estarás", "estará", "estaremos", "estareis", "estarão",
              "estaria", "estarias", "estaríamos", "estaríeis", "estariam",
              "esteja", "estejas", "estejamos", "estejais", "estejam",
              "estivesse", "estivesses", "estivéssemos", "estivésseis", "estivessem",
              "estiver", "estiveres", "estivermos", "estiverdes", "estiverem"

              "ter", "tido", "tendo",
              "tenho", "tens", "tem", "temos", "tendes", "têm",
              "tinha", "tinhas", "tínhamos", "tínheis", "tinham",
              "tive", "tiveste", "teve", "tivemos", "tivestes", "tiveram",
              "tivera", "tiveras", "tivéramos", "tivéreis", "tiveram",
              "terei", "terás", "terá", "teremos", "tereis", "terão",
              "teria", "terias", "teríamos", "teríeis", "teriam",
              "tenha", "tenhas", "tenha", "tenhamos", "tenhais", "tenham",
              "tivesse", "tivesses", "tivéssemos", "tivésseis", "tivessem",
              "tiver", "tiveres", "tivermos", "tiverdes", "tiverem", "tende",

              "haver", "havido", "havendo", "hei", "hás", "há", "havemos", "haveis", "hão",
              "havia", "havias", "havíamos", "havíeis", "haviam",
              "houve", "houveste", "houvemos", "houvestes", "houveram",
              "houvera", "houveras", "houvéramos", "houvéreis",
              "haverei", "haverás", "haverá", "haveremos", "havereis", "haverão",
              "haveria", "haverias", "haveríamos", "haveríeis", "haveriam",
              "haja", "hajas", "hajamos", "hajais", "hajam",
              "houvesse", "houvesses", "houvéssemos", "houvésseis", "houvessem",
              "houver", "houveres", "houvermos", "houverdes", "houverem"],
    "marcador": ["ai", "aí", "né", "i", "ih", "ah", "oh", "eh"],
    "numeros" : ['1', '2', '3', '4', '5', '6', '7', '8', '9']
    }

# Sugestões de remoções: estado, estáveis, eras, tende.

# I organized the stopwords into a dictionary, grouping them by word classes.
# This structure allows you to select relevant stopwords for your data cleaning to align with the goals of text processing.

# Organizei as stopwords em um dicionário, agrupando-as pela classe de palavras.
# Essa estrutura permite selecionar stopwords relevantes para a sua limpeza de dados de acordo com os objetivos do processamento de texto.

In [ ]:
import json

with open("stopwords.json", "w", encoding="utf-8") as json_file:
    json.dump(stopwordsdict, json_file, ensure_ascii=False, indent=4)

In [ ]:
def select_stopwords(stopwords, word_classes, remove_list = None):
  """Cria uma lista de stopwords selecionadas a partir de um dicionário.

  Argumentos:
    stopwordsdict (dict): Dicionário contendo todas as stopwords por classe de palavras

    word_classes (list): A lista de classes de palavras a serem incluídas nas stopwords finais.
    As classes podem ser:
    artigos, pronomes, preposicoes, adverbios, conjuncoes, verbos, marcadores discursivos, números

    remove_list (list, opcional): Uma lista com palavras a serem removidas das stopwords

  Retorna:
    stopwords (list): Uma lista final de stopwords.
  """

  stopwords = [word for word_class in word_classes for word in stopwordsdict.get(word_class, [])]
  # Use dict.get() para evitar KeyError caso a classe não se encontre no dicionário

  # Remove palavras selecionadas da list de stopwords caso o argumento remove_list seja preenchido
  # com uma lista de remoção

  if remove_list:
      stopwords = [word for word in stopwords if word not in remove_list]

  return stopwords

In [ ]:
stopwords = select_stopwords(stopwordsdict, ["artigos", "pronomes", "preposicoes", "adverbios", "conjuncoes", "marcador", "numeros"])

In [ ]:
# Pontuações e símbolos presentes nos arquivos de acordo com as diretrizes de
# transcrição do DeG

punctuation = ", . / \ [ ] ( ) : ; ? ! � … - \x94"

In [ ]:
def split_blocos(text):
  """Separa os blocos de texto em listas para que sejam recuperáveis depois do
  tratamento sintático.

  Recebe: text (str): O texto em txt.

  Retorna: Texto original (list): Uma lista com os blocos de texto
  separados."""

  # Verifica se o texto já está dividido em blocos
  if isinstance(text, list):
    return text  # Se já for uma lista, retorna como está
  else:
    return text.split('\n\n')  # Se for string, divide em blocos

In [ ]:
rio_grande = split_blocos(rio_grande)
rio_grande = [lst for lst in rio_grande if lst]

In [ ]:
import copy

# Cria uma deep copy do dataset para manter os originais
riogrande = copy.deepcopy(rio_grande)

In [ ]:
def tokenization(text, punctuation):
    """Tokeniza as palavras em um texto a partir dos espaços em branco e dos separadores.

    Recebe:
        text (list): Lista de strings com o texto a ser tokenizado.
        punctuation (str): String contendo os separadores presentes no texto.

    Retorna:
        list: Uma lista de listas com os tokens (palavras) presentes em cada bloco de texto.
    """
    # Cria uma tabela de tradução para substituir os separadores por espaços
    translation_table = str.maketrans(punctuation, " " * len(punctuation))

    # Aplica a tradução e tokeniza cada linha
    return [line.translate(translation_table).split() for line in text]

In [ ]:
def preprocessing(text, punctuation):
  """ Executa o pré-processamento de um texto.

  Recebe: uma lista com o texto e uma lista de caracteres separadores.

  Retorna: uma lista de palavras pré-processadas referentes ao texto passado.
  """

  tokens = tokenization(text, punctuation)

  tokens_normais = [[token.lower() for token in sublist] for sublist in tokens]

  return tokens_normais

In [ ]:
# print("Pré-processamento da Base Rio Grande:")
riogrande = preprocessing(rio_grande, punctuation)
# print(riogrande)
# print(len(riogrande))

In [ ]:
riogrande_sem_stopwords = [[word for word in sublist if word not in stopwords] for sublist in riogrande]

---

## Abordagens de lematização


1.   **Lematização por aprendizado de máquina**: spaCy modelo pt_core_news_sm
2.   **Lematização por depêndencias universais**: Simplemma UD-PT-GSD
3.   **Lematização por arquivos lexicográficos**: PortiLexicon-UD (Projeto POeTiSA)


## Importação do modelo do português do spaCy

In [ ]:
import spacy
!python -m spacy download pt_core_news_sm --quiet

nlp = spacy.load('pt_core_news_sm')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 64.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### Lematização por modelo e UD: spaCy

In [ ]:
def lemmatizationspacy(tokens):
  """ Lematiza um texto usando a bibliotecaspaCy.

  Recebe: uma lista de tokens (str).

  Retorna: uma lista com os tokens lematizados.
  """
  text = ' '.join(tokens)  # Une os tokens em uma única string
  doc = nlp(text)

  lemmas = [token.lemma_ for token in doc]

  return ' '.join(lemmas)

In [ ]:
# Criar DataFrame
riograndelem = pd.DataFrame({'Texto': riogrande})

# Aplicar lematização aos dados
riograndelem['Texto Lematizado'] = riograndelem['Texto'].apply(lemmatizationspacy)

In [ ]:
spacy_lem = [riograndelem[i] for i in range(len(riograndelem))]
spacy_lem = [s.lower() for s in spacy_lem if isinstance(s, str)]

In [ ]:
# Criar DataFrame
riograndelem2 = pd.DataFrame({'Texto': riogrande_sem_stopwords})

# Aplicar lematização aos dados
riograndelem2['Texto Lematizado'] = riograndelem2['Texto'].apply(lemmatizationspacy)

In [ ]:
# Exporta os dataframes da lematização em um arquivo .csv
riograndelem.to_csv(r"lematizacao_spacyRI.csv")
riograndelem2.to_csv(r"lematizacao_spacyRI_sem_stopwords.csv")

### Lematização com [Simplemma](https://github.com/adbar/simplemma)

In [ ]:
!pip install git+https://github.com/adbar/simplemma

  Cloning https://github.com/adbar/simplemma to /tmp/pip-req-build-l891p2ou
  Running command git clone --filter=blob:none --quiet https://github.com/adbar/simplemma /tmp/pip-req-build-l891p2ou
  Resolved https://github.com/adbar/simplemma to commit be7435ece1093b64bab01214d0770f8a78e7baf5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for simplemma: filename=simplemma-1.1.2-py3-none-any.whl size=67154856 sha256=9518e240e5a3ba2905f3f268678477f5c420140324c3efeeaef003e3b12d1bb9
  Stored in directory: /tmp/pip-ephem-wheel-cache-70078r0y/wheels/f8/a6/06/b7cdede3f6ff48350da0e1d78f5c1f948067ad45f092e36949
Successfully built simplemma


In [ ]:
import simplemma

def simplemmatization(texto):
  """ Lematiza um texto através da função token.lemma_ do spaCy.

  Recebe: um texto (string).

  Retorna: um dataframe com o texto original e o texto lematizado.
  """

  lemmas = []
  for token in simplemma.lemmatize(texto, lang = 'pt'):
    lemmas.append(token)
  return ''.join(lemmas)

In [ ]:
# Criar DataFrame
simlematizacao = pd.DataFrame({'Texto': [item for sublist in riogrande for item in sublist]})

# Aplicar lematização aos dados
simlematizacao['Texto Lematizado'] = simlematizacao['Texto'].apply(simplemmatization)

In [ ]:
# Criar DataFrame
simlematizacao2 = pd.DataFrame({'Texto': [item for sublist in riogrande_sem_stopwords for item in sublist]})

# Aplicar lematização aos dados
simlematizacao2['Texto Lematizado'] = simlematizacao2['Texto'].apply(simplemmatization)

In [ ]:
sim_lematizacao = simlematizacao['Texto Lematizado'].values.tolist()

In [ ]:
# Exporta o dataframe da lematização em um arquivo .csv
simlematizacao.to_csv(r"simlematizacaoRI.csv")
simlematizacao2.to_csv(r"simlematizacaoRI_sem_stopwords.csv")

### Lematização com PortiLexicon-UD

In [ ]:
portilexicon = pd.read_csv('portilexicon-ud - portilexicon-ud.csv')

In [ ]:
def lematizacao_portilexicon(texto, portilexicon = portilexicon):
    """Lematiza um texto através do documento lexicográfico PortiLexicon.

    Recebe:
        texto (list): Lista de tokens (strings) do texto.
        portilexicon (pd.DataFrame): DataFrame contendo as colunas 'a' (token) e 'a.1' (lemma).

    Retorna:
        list: Lista de tokens lematizados.
    """

    # Criar um dicionário para busca rápida dos lemas
    lemma_dict = dict(zip(portilexicon['a'], portilexicon['a.1']))

    # Dá flat na list comprehension antes do processamento
    flat_texto = [item for sublist in texto for item in sublist]

    if isinstance(flat_texto[0], list):
        flat_texto = [item for sublist in flat_texto for item in sublist]

    # Substituir tokens pelo lema correspondente ou manter o token original
    return [lemma_dict.get(token, token) for token in flat_texto]

In [ ]:
port_lem = lematizacao_portilexicon(riogrande)

In [ ]:
port_lem2 = lematizacao_portilexicon(riogrande_sem_stopwords)

In [ ]:
# Criar DataFrame
port_lem_df = pd.DataFrame({'Texto': [item for sublist in riogrande for item in sublist]})

# Aplicar lematização aos dados
port_lem_df['Texto Lematizado'] = port_lem

In [ ]:
# Criar DataFrame
port_lem_df2 = pd.DataFrame({'Texto': [item for sublist in riogrande_sem_stopwords for item in sublist]})

# Aplicar lematização aos dados
port_lem_df2['Texto Lematizado'] = port_lem2

In [ ]:
# Exporta o dataframe da lematização em um arquivo .csv
port_lem_df.to_csv(r"simlematizacaoRI.csv")
port_lem_df2.to_csv(r"simlematizacaoRI_sem_stopwords.csv")

## Validação

In [ ]:
testeriogrande = [riogrande[:1000]]
testeriogrande

In [ ]:
lematizacaomodelo = ['informante', 'de', 'ensino', 'superior', 'informante', '1', 'joão', 'carlos', 'sexo', 'masculino', 'idade', '28', 'ano', 'data', 'de', 'coleta', 'oral', '20', '11', '93', 'escrita', '21', '11', '93', 'e',
'22', '11', '93', 'parte', 'oral', 'narrativa', 'de', 'experiência', 'pessoal', 'e', 'joão', 'ter', 'algum', 'história', 'que', 'ter', 'acontecer', 'contigo', 'que', 'tu', 'querer', 'eu', 'contar', 'i',
'ter', 'um', 'recente', 'eh', 'pegar', 'o', 'lancha', 'aí', 'ir', 'eu', 'e', 'dois', 'amigo', 'meu', 'que', 'trabalhar', 'comigo', 'lá', 'o', 'edson', 'e', 'o', 'miguel', 'aí',
'entrar', 'em', 'lancha', 'né', 'ter', 'não', 'saber', 'se', 'você', 'saber', 'como', 'ser', 'que', 'o', 'lancha', 'ter', 'quatro', 'um', 'fileira', 'de','quatro', 'cadeira','assim',
'de', 'madeira', 'com', 'encosto', 'e', 'o', 'cadeira', 'ser', 'normal', 'aí', 'até', 'ir', 'entrar', 'conversar', 'aí','pedir', 'pra', 'dar', 'lugar', 'pra', 'outro', 'sentar', 'também', 'sentar', 'em', 'canto', 'o', 'outro', 'dois', 'sentar',
'perto','de', 'eu', 'só', 'que', 'eu', 'não', 'ver', 'que', 'estar', 'sem', 'encosto', 'o', 'cadeira', 'eu', 'sentar', 'o','cadeira', 'ser', 'de', 'ser', 'imóvel', 'vir', 'pra', 'trás', 'cair', 'deitado', 'em', 'colo', 'de',
'negrão', 'atrás', 'de', 'eu', 'riso', 'aquilo', 'ali', 'ficar', 'o', 'vale', 'de', 'ano', 'lá', 'em', 'em', 'narrativa', 'recontado', 'e', 'estar', 'joão',  'e', 'ter', 'algum', 'história', 'que', 'alguém',
'ter', 'te', 'contar', 'e', 'que', 'tu', 'não', 'ter', 'participado', 'e', 'que', 'tu', 'saber', 'através', 'de', 'alguém', 'i', 'eh', 'eu', 'não', 'saber', 'se', 'chegar', 'a', 'ser', 'o', 'que', 'tu', 'estar', 'querer',
'mas', 'isso', 'acontecer', 'sexta', 'feira', 'também', 'ser', 'mais', 'pra',  'mais', 'um', 'vale', 'datar', 'o', 'cara', 'chegar', 'estar', 'eh', 'almoçar', 'o', 'mãe', 'dele', 'ligar', 'pra', 'ele', 'assim', '\x93está',
'com', 'um', 'pé', 'de', 'cada','sapato','aí','o','o','colega','chegar','e','disse','\x93ué','pé','de','cada','sapato','você','achar','que','eu','ser','louco','por','um','pé','de','sapato','um','pé','de',
'cada','sapato','olhar','pra','baixo','um','sapato','com','um','lista','e','outro','sem','lista','riso','de','e','ir','trabalhar','com','um','pé','de','cada','sapato','pegar','em','pé','dele','lá','descrição',
'de','local','e','tá','joão','eh','ter','algum','lug','um','lugar','ou','algum','coisa','que','tu','querer','eu','descrever','i','eu','poder','você','descrever','o','agência','lá','em','em','norte','onde',
'eu','trabalhar','eh','logo','em','seguida','qur','tu','descer','do','lancha','ter','o','o','rua','de','frente','ali','álvaro','costa','o','agência','ser','bem','em','frente','ser','dois','piso','né','o',
'setor','de','baixo','ser','onde','funcionar','o','o','caixa','de','gerência','e','em','cima','o','pessoal','mais','o','pessoal','de','de','aplicação','ser','um','um','agência','pequeno','mas','eh','bem',
'bonito','até','relato','de','procedimento','e','tá','e','ter','algum','coisa','que','tu','querer','eu','ensinar','a','fazer','explicar','explicar','como','faz','algum','coisa','pode','ser','algum','coisa',
'relacionado','com','o','você','serviço','algum','i','ser','ser','difícil','ir','dizer','você','ensinar','a','trabalhar','em','caixa','o','o','rapaz','que','fazer','compensação','por','exemplo','quando',
'você','chegar','com','um','depósito','com','cheque','de','outro','banco','tu','tu','tem','que','carimbar','o','cheque','né','aí','em','final','de','dia','tu','tu','somar','ele','banco','por','banco','que',
'ficar','só','pra','pra','compensação','eh','ser','coisa','entender','que','eu','poder','até','você','explicar','mas','não','relato','de','opinião','e','aí','tu','querer','dar','um','opinião','sobre',
'algum','coisa','algum','assunto','i','eh','como','eu','trabalhar','em','banco','agora','que','falar','tudo','agora','sobre','o','sobre','banco','ir','você','dar','um','motivo','um','opinião','sobre',
'não','sei','aplicação','eu','achar','que','pelo','menos','ser','de','se','esperar','como','o','taxa','estar','muito','alta','em','aplicação','dever','ter','um','mudança','em','plano','econômico','de',
'de','país','pro','final','de','ano','até','estar','falar','lá','em','algum','pacote','algum','coisa','porque','a','taxa','estar','muito','al','muito','alta','muito','acima','de','inflação','o','inflação',
'que','ele','divulgar','né','então','por','a','taxa','estar','tão','alta','certamente','dever','ter','algum','mudança','em','em','plano','econômico','e','tá','i','bom','dia','e','obrigado','parte','escrito',
'narrativa','de','experiência','pessoal','ir','descrever','um','acontecimento','engraçado','que','se','passar','comigo','em','lancha','para','são','josé','do','norte','quando','ia','para','o','serviço',
'a','lancha','noivo','de','mar','ter','fileira','com','4','banco','de','madeira','e','o','assento','ser','móvel','entrar','em','lancha','juntamente','com','dois','colega','de','serviço','e','sentei','em',
'cadeira','de','canto','porém','não','ver','que','o','cadeira','estar','sem','encosto','cair','com','tudo','pra','trás','em','colo','de','um','senhor','que','estar','no','banco','de','trás','ser','o',
'vale','do','ano','narrativa','recontado','outro','acontecimento','engraçado','acontecer','outro','dia','com','um','colega','em','serviço','o','mãe','dele','ligar','pra','ele','perguntar','se','ele','estar',
'com','um','pé','de','cada','sapato','ele','ainda','ficar','bravo','dizer','que','não','estar','louco','e','olhar','para','o','pé', 'estar','mesmo','com','um','pé','de','cada','imaginar','o','situação',
'descrição','de','local','ir','descrever','o','agência','de','caixa','econômico','federal','em','são','josé','do','norte','ser','logo','em','rua','em','frente','o','hidroviário','um','agência','de','dois',
'andar','onde','em','parte','inferior','ficar','localizado','o','gerência','o','abertura','de','conta','e','o','caixa','em','andar','superior','ficar','o','cozinha','e','o','banheiro','o','almoxarifado',
'e','o','setor','de','contabilidade','e','aplicação','ser','um','agência','pequeno','porém','bem','agradável','relato','de','procedimento','ir','dar','um','pequeno','procedimento','sobre','o','tratamento',
'dado','a','cheque','de','outro','banco','recebido','pelo','caixa','em','depósito','o','caixa','recebe','o','depósito','em','cheque','carimbar','o','cheque','com','o','carimbo','de','compensação','em',
'verso','e','com','o','de','cruzamento','em','frente','verificar','o','prazo','de','comp','bloqueio','e','autenticar','o','depósito','em','final','de','expediente','fazer','o','soma','de','cheque','verificar',
'se','coincidir','com','o','total','de','depósito','recebido','e','encaminhar','para','o', 'retaguarda','relato', 'de', 'opinião', 'ir', 'dar','um', 'opinião','sobre','o','situação', 'econômico', 'que',
'provavelmente', 'dever', 'ter', 'mudança', 'em', 'final', 'desse', 'ano', 'visto', 'que', 'o', 'taxa', 'de', 'juro','em', 'aplicação', 'em', 'cdb', 'rdb', 'poupança', 'e','outro','estar','muito','elevado',
'em','relação','o', 'inflação', 'divulgado', 'achar', 'que', 'certamente', 'esse', 'equilíbrio', 'provocar', 'mudança', 'e', 'quem', 'saber', 'um', 'novo','pacote', 'econômico', 'informante', '2',
'lisandra','sexo','feminino','idade', '23','ano','data','de','coleta','oral','28','09','93','escrita','06','11','93','parte','escrito','narrativa','de','experiência','pessoal','e','lisandra', 'um',
'experiência', 'um', 'narrativa', 'teu', 'de', 'experiência']

In [ ]:
def contaacuracia(lematizacaomodelo, lematizacao):

  corretos = 0

  for i in range(len(lematizacaomodelo)):
    if lematizacaomodelo[i] == lematizacao[i]:
      corretos += 1

  acuracia = corretos / len(lematizacaomodelo)

  return acuracia

In [ ]:
contaacuracia(lematizacaomodelo, spacy_lem)

0.802

In [ ]:
contaacuracia(lematizacaomodelo, sim_lematizacao)

0.825

In [ ]:
contaacuracia(lematizacaomodelo, port_lem)

0.802

---